# Setup Environment

In [ ]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

import sys, os, shutil

sys.path.append('/usr/local/lib/python3.7/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'
for so in ['cudf', 'rmm', 'nccl', 'cuml', 'cugraph', 'xgboost', 'cuspatial']:
  fn = 'lib'+so+'.so'
  source_fn = '/usr/local/lib/'+fn
  dest_fn = '/usr/lib/'+fn
  if os.path.exists(source_fn):
    print(f'Copying {source_fn} to {dest_fn}')
    shutil.copyfile(source_fn, dest_fn)
if not os.path.exists('/usr/lib64'):
    os.makedirs('/usr/lib64')
for so_file in os.listdir('/usr/local/lib'):
  if 'libstdc' in so_file:
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib64/'+so_file)
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib/x86_64-linux-gnu/'+so_file)

!pip install -U cffi
!wget https://vcu-dapt-615.s3.amazonaws.com/properties_2016.csv.zip
!unzip properties_2016.csv.zip
!wget https://vcu-dapt-615.s3.amazonaws.com/data_dict.csv

# Analysis

## Load Data

In [ ]:
import cudf
import pandas as pd

Before we load the data, let's check our GPU usage memory:

Let's first load the data into local memory using `pandas`, and check the starting gpu memory usage.

In [ ]:
%%time
pd_data = pd.read_csv('properties_2016.csv', low_memory=False)
pd_data = pd_data.drop([
                        'propertycountylandusecode',
                        'propertyzoningdesc',
                        'taxdelinquencyflag', 
                        'taxamount', 
                        'structuretaxvaluedollarcnt', 
                        'landtaxvaluedollarcnt'
                        ], axis=1)

In [ ]:
!nvidia-smi

Now let's put the data into gpu memory by creading a `cuDF` `DataFrame`, and then check our gpu memory utilization:

In [ ]:
data = cudf.from_pandas(pd_data)

In [ ]:
!nvidia-smi

**Question 1**: How much memory (in MiB) does the `DataFrame` consume?

[PLEASE TYPE YOUR ANSWER HERE]

## Data Inspection

Now that we have our data loaded, let's see if we can answer some quick questions about the data. We'll also try to take a look at the performance differences between `pandas` and `cuDF` `DataFrames`. 

Let's start with some simple some simple details about the data. Answer the following questions below. Note that the column descriptions can be found in the file `data_dict.csv`, and loaded with `lookup = pd.read_csv('data_dict.csv')`

**Question 2**: What's the mean value of the properties (from tax assessment) in dollars for the data set? Are there any assessed values that stand out to you as unusual?

**Question 3**: How many columns are in `DataFrame`? Do any of the columns have `Null` values? Should we just remove columns/rows with nulls in them?

**Question 4**: How many different zip codes are the properties located in? How many different cities? (look at the `regionid*` fields for this, e.g. `regionidcity`)

**Question 5**: Which `regionidcity` has the most proprties, and what is the min, mean, median, max values for properties there?

In [ ]:
# Question 2 code here
data.taxvaluedollarcnt.describe()

In [ ]:
# Question 3 code here
print(data.shape)
print(data.isnull().any())

In [ ]:
# Question 4 code here
print(data.regionidzip.nunique())
print(data.regionidcity.nunique())

In [ ]:
# Question 5 code here
data.groupby('regionidcity')['parcelid'].count().sort_values(ascending=False).head()


In [ ]:
data[data['regionidcity'] == 12447.0]['taxvaluedollarcnt'].describe()

We saw above that we have some `Null` in the data. We'd like to fill in these values with the mean value for each column. 

**Question 6**: Write a `for` loop to loop through each of the columns in the `DataFrame`, compute the mean of that column with the `mean()` `Series` method, then use the `fillna()` `Series` method to fill `Null` values in that column with the mean value.

In [ ]:
%%time
for column in data.columns:

    # Check to see if the column has any null values
    if not data[column].isnull().any():
      continue
    # Compute the mean for the column
    # Question 6 code here
    mean = data[column].mean()

    # use `fillna` to fill the null values in the column with the mean
    # Question 6 code here
    data[column] = data[column].fillna(mean)

**Question 7**: Verify that none of the columns have any nulls in them.

*Hint: try `.isnull()` and `.any()` methods*

In [ ]:
# Question 7 code here
data.isnull().any()

# Modeling

In [ ]:
from cuml.ensemble import RandomForestRegressor
from cuml.preprocessing.model_selection import train_test_split
from cuml.metrics import mean_absolute_error
import numpy as np

# Convert the data types so they play well with cuML
data = data.astype(np.float32)

**Question 8**: Create a train/test split (with say 80% of the data in the training set), with the `taxvaluedollaramount` as the target variable. 

Name the outputs `X_train, X_test, y_train, y_test`

In [ ]:
# Question 8 code here
X_train, X_test, y_train, y_test = train_test_split(data, 'taxvaluedollarcnt')

**Question 9**: Create a `RandomForestRegressor` object and fit your training data

In [ ]:
# Question 9 code here
clf = RandomForestRegressor(n_estimators=40,
                   max_depth=16,
                   max_features=1.0)

clf.fit(X_train, y_train)

**Question 10**: Make predictions on the test data and calculate the mean absolute error

In [ ]:
# Question 10 code here
y_pred = clf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(mae)